# Insider Trading Analysis - SEC Edgar

## Boilerplate

Let's start by defining some boilerplate to define global variables.

# Download Form 4 Data

Let's implement the download and XML parse methods. The XML parser extracts all non-derivative transactions listed in the 4 filing. It considers `disposal` and `acquisition` of items and calculates the sum of all transactions (sold and bought).

**Important:** the downloader doesn't differentiate between filing issuer and reporter. In other words, we download the issuer's and reporter's filing. The reported transactions in both filings are the same and we double count transactions. That's OK for demonstration purposes but has to be cleaned when building a production version.

In [10]:
files = []
data = []
prices = []
form = "4"
ticker="SCHW"
year= "2022"
month= "01"
day= "01"

In [11]:
from sec_edgar_downloader import Downloader
dl = Downloader()

#TODO: calculate dates missing based on files saved locally

dl.get(form, ticker, after=year+'-'+month+'-'+day)

75

## Compressing Filenames

Adding filename to filings open XML data

In [12]:
import os
def compress_filings (filings):
    compressed_filings = []
    for filingDir in filings:
        fullSubmissionFname = os.path.join(path_form, filingDir, "filing-details.xml")
        compressed_filings.append(fullSubmissionFname)
    return compressed_filings

In [13]:
import os, re
pattern = re.compile("([0-9]+)")
path_form = os.path.join("sec-edgar-filings", ticker, '4')

files = compress_filings(os.listdir(path_form))
# files[0:1]

# Open XML Files

In [14]:
import xml.etree.ElementTree as ET
import re

def open_xml(file):
    xml_file=  open(file)
    data = xml_file.read()
    xml_file.close()
    matcher = re.compile('<\?xml.*ownershipDocument>', flags=re.MULTILINE|re.DOTALL)
    matches = matcher.search(data)
    # the first matching group is the extracted XML of interest
    xml = matches.group(0)
    # instantiate the XML object
    root = ET.fromstring(xml)
    return root

In [15]:
# Calculate the total transaction amount in $ of a giving form 4 in XML
def flatten_info(xml):

    if xml is None:
        return
    try:
        periodOfReport = xml.findall("./periodOfReport")[0].text
        rptOwnerName = xml.findall("./reportingOwner/reportingOwnerId/rptOwnerName")[0].text
        issuerTradingSymbol = xml.findall("./issuer/issuerTradingSymbol")[0].text
        
        isOfficer = xml.find("./reportingOwner/reportingOwnerRelationship/isOfficer")

        nonDerivativeTransactions = xml.findall("./nonDerivativeTable/nonDerivativeTransaction")
        for t in nonDerivativeTransactions:
            # P is purchase and S is sell
            generalCode = t.find('./transactionCoding/transactionCode').text
            # D for disposed or A for acquired
            action = t.find('./transactionAmounts/transactionAcquiredDisposedCode/value').text
            # number of shares disposed/acquired
            shares = t.find('./transactionAmounts/transactionShares/value').text
            # price
            priceRaw = t.find('./transactionAmounts/transactionPricePerShare/value')
            price = 0 if priceRaw is None else priceRaw.text
            # set prefix to -1 if derivatives were disposed. set prefix to 1 if derivatives were acquired.
            prefix = -1 if action == 'D' else 1
            # calculate transaction amount in $
            amount = prefix * float(shares) * float(price)
            data.append({
                'periodOfReport': periodOfReport,
                'generalCode': generalCode,
                'issuerTradingSymbol': issuerTradingSymbol,
                'rptOwnerName': rptOwnerName,
                'action': action, 
                'shares': shares,
                'amount': amount,
                'price': price,
                'isOfficer': 0 if isOfficer is None else isOfficer.text

            })
         
    except Exception as exception:
        print(exception, xml)
    

In [16]:
# FOR TESTING
# data = []
# xml = open_xml(files[0])
# flatten_info(xml)
# data[:1]

In [17]:
# Get the XML file for each filing
# Calculate the total transaction amount per filing
# Save the calculate transaction values to the data dict
for filing in files:
    xml = open_xml(filing)
    flatten_info(xml)
# Running the function prints the URL of each filing fetched
data[:1]

[{'periodOfReport': '2021-03-01',
  'generalCode': 'M',
  'issuerTradingSymbol': 'SCHW',
  'rptOwnerName': 'Bettinger Walter W',
  'action': 'A',
  'shares': '57815.0000',
  'amount': 3685706.25,
  'price': '63.7500',
  'isOfficer': '1'}]

In [20]:
data.to_json('schw.json', orient='records')

AttributeError: 'list' object has no attribute 'to_json'

# Data Plot
Displays insider trading in graph.

In [ ]:
import pandas as pd
data = pd.json_normalize(data)

In [ ]:
data.periodOfReport.min(), data.periodOfReport.max()

In [ ]:
data.rptOwnerName.value_counts().head()

In [ ]:
data.generalCode.value_counts()

In [ ]:
# data.head()
buys = data[data.generalCode == 'P'] 
sells = data[data.generalCode == 'S']
officers_buy = data.query('generalCode == "P" and isOfficer == "1"')
officers_buy.head()

In [ ]:
import yfinance as yf

start_date = "{}-{}-{}".format(year, month, day)
prices = yf.download(ticker, start=start_date)
prices.head()


In [ ]:
import numpy as np
import pandas as pd

from bokeh.plotting import figure
from bokeh.io import show, output_notebook, reset_output

# importing the modules
reset_output()
output_notebook()

# instantiating the figure object
graph = figure(x_axis_type = "datetime", title = "Stock Closing Prices", tools='save,pan,box_zoom,reset,wheel_zoom,hover')
graph.xaxis.axis_label = 'Date'
graph.yaxis.axis_label = 'Price (in USD)'

def datetime(x):
    return np.array(x, dtype=np.datetime64)
    
# plotting the line graph 
graph.line(datetime(prices.index), prices['Adj Close'], color='#A6CEE3', legend_label=ticker)

# plotting insider trading
graph.circle(datetime(officers_buy.periodOfReport), buys.price, size = 10, color = 'green')
graph.circle(datetime(sells.periodOfReport), sells.price, size = 10, color = 'red')

# displaying the model
show(graph)

metric in time window
- trend Q to year
- how do we evaluate the trend?

# Get Insider Trading Activity

In [ ]:
def getBins (filings):
    bins = {}
    for index, row in filings.iterrows():
        filedAt = row['periodOfReport']
        nonDerivativeTransactions = row['amount']
        value = bins[filedAt] + nonDerivativeTransactions if filedAt in bins else nonDerivativeTransactions
        bins[filedAt] = round(value, 2)
    return bins

In [ ]:
bins = getBins(data)
# bins

In [ ]:
import matplotlib.pyplot as plt
import matplotlib

# Set size of figure
plt.rcParams['figure.figsize'] = [150, 100]

# Prettify y axis: 2000000 to $2M
def millions(x, pos):
    return '${:,.0f}M'.format(x*1e-6)

fig, ax = plt.subplots()
# Define bar plot
ax.bar(range(len(bins)), list(bins.values()), align='center')
ax.grid(True)
ax.yaxis.set_major_formatter(matplotlib.ticker.FuncFormatter(millions))
# Prettify x axis
fig.autofmt_xdate()
# Set x axis values
plt.xticks(range(len(bins)), list(bins.keys()))
plt.show()